In [1]:
import numpy as np
from scipy.spatial import KDTree
from multiprocessing import Pool,cpu_count
from qscore_utils import (
          sphere_points,
          trilinear_interpolation,
          rowwise_corrcoef,
          )
# from scipy.spatial.distance import pdist, cdist
# from cctbx.array_family import flex



# def radial_shell_worker(args):
#     i,atoms_xyz,n_probes,radius_shell,rtol, selection= args

    
#     kdtree_atoms = KDTree(atoms_xyz)
#     # manage selection
#     if selection is None:
#         selection =  np.full(len(atoms_xyz),True)
#     else:
#         selection = np.array(selection)
        
        
#     n_atoms = np.sum(selection)
#     atoms_xyz_sel = atoms_xyz[selection]
#     print(atoms_xyz_sel)
#     probe_xyz_r = sphere_points(atoms_xyz_sel,radius_shell,n_probes)
    
#     # query kdtree to find probe-atom interactions (the slowest part by far)
#     counts = kdtree_atoms.query_ball_point(probe_xyz_r,radius_shell*rtol,return_length=True) #  (n_atoms,n_probes) (a count value for each probe)
#     d = sorted(list(set(cdist(atoms_xyz,probe_xyz_r).ravel())))
#     print(d)
#     # each value in counts is the number of atoms within radius+tol of each probe
    
#     # Only want to select probes with a single atom neighbor
#     keep_sel = counts==1

#     return (probe_xyz_r, keep_sel)

# def radial_shell_mp(atoms_xyz,n_shells=21,n_probes=64,radii=None,rtol =1.1,num_processes=4,selection=None):


#     radii = []
#     for r in list(radii):
#         if r==0:
#             r=1e6
#         radii.append
#     # Create argument tuples for each chunk
#     args = [(i,atoms_xyz,n_probes,radius_shell,rtol,selection) for i,radius_shell in enumerate(radii)]

#     # Create a pool of worker processes
#     with Pool(num_processes) as p:
#         # Use the pool to run the trilinear_interpolation_worker function in parallel
#         results = p.map(radial_shell_worker, args)

#     # stackthe results from each process
#     probe_xyz = np.stack([result[0] for result in results])
#     keep_mask = np.stack([result[1] for result in results])
#     return probe_xyz,keep_mask



In [2]:
# from qscore_cctbx import query_atom_neighbors, query_atom_neighbors, query_ball_point_flex
# from itertools import chain

# from scitbx_array_family_flex_ext import vec3_double, double, size_t
# def cdist_flex(A,B):
#     n_a = A.focus()[0]
#     n_b = B.focus()[0]
#     assert isinstance(A,double) and isinstance(B,double), "Provide a double array with shape, but not a vec3_double"
   

#     def indices_2d_flex(dimensions):
#         N = len(dimensions)
#         if N != 2:
#             raise ValueError("Only 2D is supported for this implementation.")

#         # Create the row indices
#         row_idx = flex.size_t(chain.from_iterable([[i] * dimensions[1] for i in range(dimensions[0])]))

#         # Create the column indices
#         col_idx = flex.size_t(chain.from_iterable([list(range(dimensions[1])) for _ in range(dimensions[0])]))

#         return row_idx, col_idx

#     #print(A.as_numpy_array().shape,B.as_numpy_array().shape)

#     i_idxs, j_idxs = indices_2d_flex((n_a,n_b))
#     xi = i_idxs*3
#     yi = i_idxs*3 + 1
#     zi = i_idxs*3 + 2

#     xa = A.select(xi)
#     ya = A.select(yi)
#     za = A.select(zi)

#     xj = j_idxs*3
#     yj = j_idxs*3 + 1
#     zj = j_idxs*3 + 2


#     xb = B.select(xj)
#     yb = B.select(yj)
#     zb = B.select(zj)

#     d = ((xb - xa)**2 + (yb - ya)**2 + (zb - za)**2)**0.5

#     d.reshape(flex.grid(n_a,n_b))

#     return d

# def query_ball_point_flex(A,B,radius):
    
#     A_vec3_input = False
#     B_vec3_input = False
#     if isinstance(A,vec3_double):
#         n_a = A.size()
#         A = A.as_1d().as_double()
#         A_vec3_input = True
#     else:
#         n_a = A.focus()[0]

#     if isinstance(B,vec3_double):
#         n_b = B.size()
#         B = B.as_1d().as_double()
#         B_vec3_input = True
#     else:
#         n_b = B.focus()[0]

        
        

#     # Calculate the distance from all points to the query point
#     distances = cdist_flex(A, B)
#     sel = distances < radius
#     #print(sel.focus())
#     #print(n_a,n_b)
#     out = []
#     for i in range(n_b):
#         c = 0
#         for j in range(n_a):
#             if sel[j,i]:
#                 c+=1
#         out.append(c)

#     out_np = sel.as_numpy_array().sum(axis=0)
#     assert np.all(np.array(out)==out_np)

#     return out



In [3]:
def sphere_points_np(ctr,rad,N):
    h = -1.0 + (2.0 * np.arange(N) / float(N-1))[:, np.newaxis]
    phis = np.arccos(h)
    thetas = np.zeros_like(phis)
    a = (3.6 / np.sqrt(N * (1.0 - h[1:-1]**2)))
    thetas[1:-1, :] = a
    thetas = np.cumsum(thetas,axis=0)
    x = np.sin(phis) * np.cos(thetas)
    y = np.sin(phis) * np.sin(thetas)
    z = np.cos(phis)
    points = rad * np.stack([x, y, z], axis=-1)
    points = points.reshape(1, N, 3)
    points = ctr[:, np.newaxis, :] + points # broadcast add
    return points
    

In [4]:
def sphere_points_flex(ctr,rad,N):
    h = -1.0 + (2.0 * flex.double_range(N) / (N-1))
    phis = flex.acos(h)
    thetas = flex.double(len(phis),0.0)
    a = (3.6 / flex.sqrt(N * (1.0 - h[1:-1]**2)))
    thetas = thetas.set_selected(flex.size_t_range(1,N-1),a)
    
    # cumulative sum operation
    def cumsum_flex(arr):
        # should perform as np.cumsum
        result = []
        running_sum = 0.0
        for i,x in enumerate(arr):
            running_sum += x
            result.append(running_sum)
        return flex.double(result)
    
    
    thetas = cumsum_flex(thetas)
    x = flex.sin(phis) * flex.cos(thetas)
    y = flex.sin(phis) * flex.sin(thetas)
    z = flex.cos(phis)
    rad = float(rad)
    points = rad * flex.vec3_double(x,y,z)
    
    # add generated points to center points
    def broadcast_add(ctr, points):

        # add and populate additional dimension. equivalent to:
        # result = ctr[:, np.newaxis, :] + points
        
        N = points.size()
        M = ctr.size()
        # Preallocate an array of shape (M*N, 3)
        result = flex.vec3_double(M*N)

        for i in range(M):
            for j in range(N):
                flat_index = i * N + j
                new_point = tuple(ctr[i:i+1] + points[j:j+1])[0]
                result[flat_index] = new_point


        result = result.as_1d().as_double()
        result.reshape(flex.grid(len(ctr),len(points),3))
        return result
    
    # apply function
    points = broadcast_add(ctr,points)
    return points

In [5]:
from iotbx.data_manager import DataManager
from cctbx.array_family import flex
import numpy as np
dm = DataManager()
dm.process_model_file("../data/6cvm.cif")
dm.process_real_map_file("../data/emd_7770.map")
mmm = dm.get_map_model_manager()
assert flex.sum(mmm.map_manager().density_at_sites_cart(mmm.model().get_sites_cart()))>0,(
    "Negative or zero density around atoms. Map/model likely not aligned")
model = mmm.model()
mm = mmm.map_manager()
M = mm.map_data()

In [6]:
model = mmm.model()
mm = mmm.map_manager()
M = mm.map_data()
n_probes = 8
selection = model.selection("chain A and resseq 1:10")
num_processes = 1
rtol = 0.9
args_np = 0,model.get_sites_cart().as_numpy_array(),n_probes,1.0,rtol, selection.as_numpy_array()
args_flex = 0,model.get_sites_cart(),n_probes,1.0,rtol, selection




In [7]:
def radial_shell_worker_v2_np(args):
    
    # unpack args
    i,atoms_xyz,n_probes,radius_shell,rtol, selection= args

    # manage selection input
    if selection is None:
        selection = np.arange(len(atoms_xyz))
    else:
        assert selection.dtype == bool

    # do selection
    n_atoms = atoms_xyz.shape[0]
    atoms_xyz_sel = atoms_xyz[selection]

    # get probe coordinates
    probe_xyz = sphere_points_np(atoms_xyz_sel,radius_shell,n_probes)
    kdtree_atoms = KDTree(atoms_xyz)
    counts = kdtree_atoms.query_ball_point(probe_xyz,radius_shell*rtol,return_length=True) #atom counts for each probe, for probes in shape (n_atoms,n_probes)
    probe_mask = counts==0 # keep probes with 0 nearby atoms. The rtol ensures self is not counted

    return probe_xyz, probe_mask

In [8]:
def radial_shell_worker_v2_flex(args):
    
    # unpack args
    i,atoms_xyz,n_probes,radius_shell,rtol, selection= args

    # manage selection input
    if selection is None:
        selection =  flex.size_t_range(len(atoms_xyz))
    else:
        assert isinstance(selection,flex_bool)


    # do selection
    n_atoms = selection.count(True)
    atoms_xyz_sel = atoms_xyz.select(selection)
    
    # get probe coordinates
    probe_xyz = sphere_points_flex(atoms_xyz_sel,radius_shell,n_probes)
    
    #get a "tree", which is just a dictionary of index:local neighbor indices
    tree, _ = query_atom_neighbors(model,radius=3.5)

    # query to find the number of atoms within the clash range of each probe
    counts =query_ball_point_flex(tree,atoms_xyz,probe_xyz,r=radius_shell*rtol)
    probe_mask = counts==0
    return probe_xyz, probe_mask

In [9]:
from scitbx_array_family_flex_ext import bool as flex_bool
from qscore_combined import sphere_points_combined
from qscore_cctbx import query_atom_neighbors, query_ball_point_flex

def radial_shell_worker_v2_combined(args):

    i,atoms_xyz,n_probes,radius_shell,rtol, selection= args
    
    # manage selection input
    if selection is None:
        selection_flex =  flex.size_t_range(len(atoms_xyz))
        selection_np = np.arange(len(atoms_xyz))
    else:
        assert isinstance(selection,flex_bool)
        selection_flex = selection
        selection_np = np.array(selection)




    # get atoms
    n_atoms = selection_flex.count(True)
    atoms_xyz_flex = atoms_xyz
    atoms_xyz_np = atoms_xyz_flex.as_numpy_array()


    # do selection
    atoms_xyz_sel_np = atoms_xyz_np[selection_np]
    atoms_xyz_sel_flex = atoms_xyz_flex.select(selection_flex)


    # get probe coordinates

    # numpy/flex
    probe_xyz_np,probe_xyz_flex = sphere_points_combined(atoms_xyz_sel_np,atoms_xyz_sel_flex,radius_shell,n_probes)
    assert np.all(np.isclose(probe_xyz_np,probe_xyz_flex.as_numpy_array()))


    # query for atom neighbors

    # numpy
    kdtree_atoms = KDTree(atoms_xyz)
    counts_np = kdtree_atoms.query_ball_point(probe_xyz_np,radius_shell*rtol,return_length=True) #atom counts for each probe, for probes in shape (n_atoms,n_probes)
    probe_mask_np = counts_np==0 # keep probes with 0 nearby atoms. The rtol ensures self is not counted


    # flex


    # get a "tree", which is just a dictionary of index:local neighbor inices
    tree, _ = query_atom_neighbors(model,radius=3.5)

    # query to find the number of atoms within the clash range of each probe
    counts_flex =query_ball_point_flex(tree,atoms_xyz_flex,probe_xyz_flex,r=radius_shell*rtol)
    probe_mask_flex = counts_flex==0
    
    
    assert np.all(probe_mask_np==probe_mask_flex.as_numpy_array())
    return probe_xyz_np, probe_xyz_flex,probe_mask_np, probe_mask_flex

In [10]:
def radial_shell_v2_mp_np(atoms_xyz,n_probes=64,radii=np.linspace(0.1,2,12),rtol=0.9,num_processes=cpu_count(),selection=None):
    
    worker_func = radial_shell_worker_v2_np
    #i,atoms_xyz,n_probes,radius_shell,rtol, selection= args
    # Create argument tuples for each chunk
    args = [(i,atoms_xyz,n_probes,radius_shell,rtol,selection) for i,radius_shell in enumerate(radii)]
    
    # Create a pool of worker processes
    if num_processes >1:
        with Pool(num_processes) as p:
            # Use the pool to run the trilinear_interpolation_worker function in parallel
            results = p.map(worker_func, args)
    else:
        results = []
        #for arg in tqdm(args):
        for arg in args:
            result = worker_func(arg)
            results.append(result)

    
    # stack numpy
    probe_xyz_all = [result[0] for result in results]
    probe_mask_all = [result[1] for result in results]
    

    probe_xyz = np.stack(probe_xyz_all)
    probe_mask = np.stack(probe_mask_all)
    
   
    return probe_xyz, probe_mask


In [28]:
import math
def radial_shell_v2_mp_flex(atoms_xyz,n_probes=64,radii=np.linspace(0.1,2,12),rtol=0.9,num_processes=cpu_count(),selection=None):
    
    worker_func = radial_shell_worker_v2_flex
    #i,atoms_xyz,n_probes,radius_shell,rtol, selection= args
    # Create argument tuples for each chunk
    args = [(i,atoms_xyz,n_probes,radius_shell,rtol,selection) for i,radius_shell in enumerate(radii)]
    
    # Create a pool of worker processes
    if num_processes >1:
        with Pool(num_processes) as p:
            # Use the pool to run the trilinear_interpolation_worker function in parallel
            results = p.map(worker_func, args)
    else:
        results = []
        #for arg in tqdm(args):
        for arg in args:
            result = worker_func(arg)
            results.append(result)
    
    # stack the results from each shell into single arrays
    probe_xyz_all = [result[0] for result in results]
    keep_mask_all = [result[1] for result in results]
    n_atoms = probe_xyz_all[0].focus()[0]
    out_shape = (n_shells,n_atoms,n_probes,3 )
    out_size = math.prod(out_shape)
    shell_size = math.prod(out_shape[1:])
    out_probes = flex.double(out_size,-1.0)
    out_mask = flex.bool(n_atoms*n_shells*n_probes,False)

    for i,p in enumerate(probe_xyz_all):
        start = i*shell_size
        stop = start+shell_size
        out_probes = out_probes.set_selected(flex.size_t_range(start,stop),p.as_1d())
    out_probes.reshape(flex.grid(*out_shape))

    for i,k in enumerate(keep_mask_all):
        start = i*(n_atoms*n_probes)
        stop = start+(n_atoms*n_probes)
        out_mask = out_mask.set_selected(flex.size_t_range(start,stop),k.as_1d())
    out_mask.reshape(flex.grid(n_shells,n_atoms,n_probes))

    return out_probes, out_mask
    

In [12]:
# test individual
probe_xyz_np, probe_mask_np = radial_shell_worker_v2_np(args_np)
probe_xyz_flex, probe_mask_flex = radial_shell_worker_v2_flex(args_flex)

assert np.all(np.isclose(probe_xyz_np, probe_xyz_flex.as_numpy_array()))
assert np.all(probe_mask_np==probe_mask_flex.as_numpy_array())

In [27]:
%pdb

Automatic pdb calling has been turned ON


In [29]:
# test individual mp
from tqdm.notebook import tqdm


  0%|          | 0/2 [00:00<?, ?it/s]

In [33]:
probe_xyz_np, probe_mask_np = radial_shell_v2_mp_np(model.get_sites_cart().as_numpy_array(),n_probes=8,radii=[0.5,1],num_processes=1)
probe_xyz_flex, probe_mask_flex = radial_shell_v2_mp_flex(model.get_sites_cart(),n_probes=8,radii=[0.5,1],num_processes=1)
assert np.all(np.isclose(probe_xyz_np, probe_xyz_flex.as_numpy_array()))
assert np.all(probe_mask_np==probe_mask_flex.as_numpy_array())

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [77]:
probe_xyz = probe_xyz_np
keep_mask = probe_mask_np
keep_mask_flex = probe_mask_flex

In [78]:
# debug
do_np = True
do_flex = True
do_test = True

# to test must do both
if do_test:
    do_np = True
    do_flex = True
# PROCEED



# flatten numpy arrays
if do_np:
    n_shells,n_atoms,n_probes,_ = probe_xyz.shape
    probe_xyz_flat = probe_xyz.reshape((n_atoms*n_shells*n_probes,3))
    keep_mask_flat = keep_mask.reshape(-1) # (n_shells*n_atoms*n_probes,)


# init cctbx arrays

if do_flex:
    n_shells,n_atoms,n_probes,_ = probe_xyz_flex.focus()
    probe_xyz_cctbx = flex.double(probe_xyz)
    keep_mask_cctbx = flex.bool(keep_mask)




# APPLY MASK BEFORE INTERPOLATION

# numpy
if do_np:
    masked_probe_xyz_flat = probe_xyz_flat[keep_mask_flat]

# cctbx
if do_flex:
    keep_mask_cctbx_fullflat = []

    for val in keep_mask_cctbx:
        for _ in range(3):  # since A has an additional dimension of size 3
            keep_mask_cctbx_fullflat.append(val)

    mask = flex.bool(keep_mask_cctbx_fullflat)
    #indices = flex.int([i for i in range(1, keep_mask_cctbx.size() + 1) for _ in range(3)])
    sel = probe_xyz_cctbx.select(mask)
    #sel_indices = indices.select(mask)
    masked_probe_xyz_flat_cctbx = flex.vec3_double(sel)


    # INTERPOLATE

# numpy
if do_np:
    M = mm.map_data()
    volume = M.as_numpy_array()
    voxel_size = np.array(mm.pixel_sizes())
    masked_density = trilinear_interpolation(volume, masked_probe_xyz_flat, voxel_size=voxel_size)

# cctbx
if do_flex:
    masked_density_cctbx = mm.density_at_sites_cart(masked_probe_xyz_flat_cctbx)

    # test equivalent
    if do_test:
        assert np.all(np.isclose(masked_density,masked_density_cctbx.as_numpy_array()))


# reshape interpolated values to (n_shells,n_atoms, n_probes)

# numpy
if do_np:
    d_vals = np.zeros((n_shells, n_atoms, n_probes))
    d_vals[keep_mask] = masked_density

# cctbx
if do_flex:
    keep_mask_cctbx.reshape(flex.grid(n_shells*n_atoms*n_probes))
    d_vals_cctbx = flex.double(keep_mask_cctbx.size(),0.0)
    d_vals_cctbx = d_vals_cctbx.set_selected(keep_mask_cctbx,masked_density_cctbx)
    d_vals_cctbx.reshape(flex.grid(n_shells,n_atoms,n_probes))

    if do_test:
        # test
        assert np.all(np.isclose(d_vals,d_vals_cctbx.as_numpy_array()))


# reshape to (M,N*L) for rowwise correlation

# numpy
if do_np:
    d_vals_2d = d_vals.transpose(1,0,2).reshape(d_vals.shape[1], -1)

# cctbx
if do_flex:
    def custom_reshape_indices(flex_array):
        N,M,L = flex_array.focus()
        result = flex.double(flex.grid(M, N * L))

        for i in range(N):
            for j in range(M):
                for k in range(L):
                    # Calculate the original flat index
                    old_index = i * M * L + j * L + k
                    # Calculate the new flat index after transpose and reshape
                    new_index = j * N * L + i * L + k
                    result[new_index] = flex_array[old_index]

        return result

    d_vals_2d_cctbx = custom_reshape_indices(d_vals_cctbx)


# test
if do_test:
    assert np.all(np.isclose(d_vals_2d,d_vals_2d_cctbx.as_numpy_array()))


# create the reference data

# numpy
if do_np:
    M = mm.map_data().as_numpy_array()
    maxD = min(M.mean()+M.std()*10,M.max())
    minD = max(M.mean()-M.std()*1,M.min())
    A = maxD-minD
    B = minD
    u = 0
    sigma = 0.6
    x = np.array(radii)
    y = A * np.exp(-0.5*((x-u)/sigma)**2) + B 




#cctbx
if do_flex:
    M = mm.map_data()
    maxD_cctbx = min(flex.mean(M)+M.standard_deviation_of_the_sample()*10,flex.max(M))
    minD_cctbx = max(flex.mean(M)-M.standard_deviation_of_the_sample()*1,flex.min(M))
    A_cctbx = maxD_cctbx-minD_cctbx
    B_cctbx = minD_cctbx
    u = 0
    sigma = 0.6
    x = flex.double(radii)
    y_cctbx = A_cctbx * flex.exp(-0.5*((flex.double(x)-u)/sigma)**2) + B_cctbx




    # test
    if do_test:
        assert np.all(np.isclose(np.array(y_cctbx),y))


# Stack and reshape data for correlation calc

# numpy
if do_np:
    # stack the reference to shape (n_shells,n_atoms,n_probes)
    g_vals = np.repeat(y[:,None],n_probes,axis=1)
    g_vals = np.expand_dims(g_vals,1)
    g_vals = np.tile(g_vals,(n_atoms,1))


    # reshape
    g_vals_2d = g_vals.transpose(1,0,2).reshape(g_vals.shape[1], -1)
    d_vals_2d = d_vals.transpose(1,0,2).reshape(d_vals.shape[1], -1)
    mask_2d = keep_mask.transpose(1,0,2).reshape(keep_mask.shape[1], -1)



# cctbx
if do_flex:
    # 1. Repeat y for n_probes (equivalent to np.repeat)
    g_vals_cctbx = [[val] * n_probes for val in y_cctbx]

    # 2. Add a new dimension (equivalent to np.expand_dims)
    g_vals_expanded = [[item] for item in g_vals_cctbx]

    # 3. Tile for each atom (equivalent to np.tile)
    g_vals_tiled = []
    for item in g_vals_expanded:
        g_vals_tiled.append(item * n_atoms)


    g_vals_cctbx = flex.double(np.array(g_vals_tiled) )


    # test 
    if do_test:
        assert np.all(np.isclose(g_vals_cctbx.as_numpy_array(),g_vals))


# # CALCULATE Q

# # numpy
if do_np:
    q = rowwise_corrcoef(g_vals_2d,d_vals_2d,mask=mask_2d)



# cctbx
if do_flex:
    d_vals_cctbx = d_vals_cctbx.as_1d()
    g_vals_cctbx = g_vals_cctbx.as_1d()
    keep_mask_cctbx_double = keep_mask_cctbx.as_1d().as_double()
    q_cctbx = []
    for atomi in range(n_atoms):

        #inds = nd_to_1d_indices((None,atomi,None),(n_shells,n_atoms,n_probes))
        inds = optimized_nd_to_1d_indices(atomi,(n_shells,n_atoms,n_probes))
        inds = flex.uint32(inds)
        d_row = d_vals_cctbx.select(inds)

        if do_test:
            assert np.all(np.isclose(d_row.as_numpy_array(),d_vals_2d[atomi]))

        g_row = g_vals_cctbx.select(inds)
        if do_test:
            assert np.all(np.isclose(g_row.as_numpy_array(),g_vals_2d[atomi]))

        mask = keep_mask_cctbx.select(inds)
        if do_test:
            assert np.all(np.isclose(mask.as_numpy_array(),mask_2d[atomi]))

        d = d_row.select(mask)
        g = g_row.select(mask)
        qval = flex.linear_correlation(d,g).coefficient()
        q_cctbx.append(qval)


    q_cctbx = flex.double(q_cctbx)
    if do_test:
        assert np.all(np.isclose(q,np.array(q_cctbx)))

ValueError: not enough values to unpack (expected 4, got 3)

In [79]:
probe_xyz.shape

(64, 8, 3)